This test Script shows how to use the library kernelo to generate learning data set, and how to use the functional model with custom photometries.

In [ ]:
# Import kernelo library
import kernelo as ker

import numpy as np
import json 
import time

In [ ]:
# Read geometries and photometries from test file
with open('test_hapke.json') as json_file:
    data = json.load(json_file)
    
    geom = np.array([data['eme'],
                     data['inc'],
                     data['phi']]).transpose() # column indexing
    
    photom = np.array([
        data['omega'],
        [x/30 for x in data['theta0']],
        data['b'],
        data['c'],
        data['b0'],
        data['hh']]).transpose() # column indexing

Create a Hapke model with its parameters adapter

In [ ]:
# Create Hapke model's adapter

# Create a six photometric parameters Hapke model adapter
mySixParamsAdapter = ker.SixParamsHapkeAdapterConfig()

# Create a four photometric parameters Hapke model adapter with b0 = 1.0 and h = 0.0
myFourParamsAdapter = ker.FourParamsHapkeAdapterConfig(1.0, 0.0)

# Create a three photometric parameters Hapke model adapter with b0 = 1.0 and h = 0.0
myThreeParamsAdapter = ker.ThreeParamsHapkeAdapterConfig(1.0, 0.0)

# Create 2002 Hapke model's version with Theta_bar_scalling = 30
my02Model = ker.HapkeModelConfig("2002", mySixParamsAdapter, geom, 30).create()

# Create 2093 Hapke model's version with Theta_bar_scalling = 30
my93Model = ker.HapkeModelConfig("1993", myFourParamsAdapter, geom, 30).create()

In [ ]:
start_time = time.time()

# Calculate reflectances 
Y = []

for p in photom:
    Y.append(my02Model.F(p))

elapsed_time = (time.time() - start_time)
print("Execution time for Y = F(X) where D = 50, L= 6 and n = 10000 : " , elapsed_time)

Data generation test

In [ ]:
# Create a Gaussian statistical model based on Hapke 1993 model with random covariance and
# a seed = 12345
covariance = np.random.uniform(0, 0.5, my93Model.get_D_dimension())
myStatModel_1 = ker.GaussianStatModelConfig("sobol", my93Model, covariance, 12345).create()

x_gen, y_gen = myStatModel_1.gen_data(10)

In [ ]:
# Create a statistical model based on Hapke 2002 model with r = 20% (noise effect) and
# a seed = 12345
myStatModel_2 = ker.DependentGaussianStatModelConfig("sobol", my02Model, 20, 12345).create()

x_gen, y_gen = myStatModel_2.gen_data(10)